In [ ]:
import os

import numpy as np
import openai
import pandas as pd



from sklearn.cluster import Birch


from sklearn.metrics import silhouette_score



from sklearn.model_selection import GridSearchCV



# Set your OpenAI API key



openai.api_key = "OPENAI_API_KEY"



# Load your data from CSV into a DataFrame


df = pd.read_csv("data\broken_summary_new_data.csv")



# Extract text data from the 'text' column



text_data = df["text"].tolist()



# Initialize an empty list to store embeddings



all_embeddings = []



# Define maximum chunk size (adjust as needed)



max_chunk_size = 500  # You can adjust this value based on your requirements



# Generate embeddings for each chunk of text


for summary in text_data:

    # Split the summary into chunks of maximum size

    chunks = [
        summary[i : i + max_chunk_size] for i in range(0, len(summary), max_chunk_size)
    ]


    # Generate embeddings for each chunk

    chunk_embeddings = []

    for chunk in chunks:

        # Generate embeddings for the chunk using Ada-002

        embedding = openai.Embed("ada-002", texts=chunk)

        chunk_embeddings.append(embedding)


    # Aggregate embeddings of chunks (e.g., by averaging)

    aggregated_embedding = np.mean(chunk_embeddings, axis=0)


    # Append the aggregated embedding to the list of all embeddings

    all_embeddings.append(aggregated_embedding)



# Convert the list of embeddings to a numpy array



embeddings = np.array(all_embeddings)



# Create Birch clustering object



birch = Birch()



# Define hyperparameters for tuning



param_grid = {"threshold": [0.1, 0.5, 1.0], "branching_factor": [50, 100, 200]}



# Perform grid search with cross-validation



grid_search = GridSearchCV(birch, param_grid, cv=5, scoring="silhouette")


grid_search.fit(embeddings)



# Get best parameters and best silhouette score


best_params = grid_search.best_params_


best_score = grid_search.best_score_



print("Best Parameters:", best_params)


print("Best Silhouette Score:", best_score)



# Fit Birch clustering with best parameters



best_birch = Birch(**best_params)


best_birch.fit(embeddings)



# Predict clusters


labels = best_birch.predict(embeddings)

In [ ]:
import pandas as pd
import requests


# Step 1: Parse CSV file into Whisper API format
def parse_transcripts_to_whisper_format(csv_file):
    # Read CSV file into DataFrame
    df = pd.read_csv(csv_file)

    # Group text by speaker
    grouped_text = (
        df.groupby("speaker_label")["text"].apply(lambda x: " ".join(x)).reset_index()
    )

    # Concatenate text for each speaker into a single string
    conversations = grouped_text["text"].tolist()

    # Format conversations for Whisper API input
    whisper_input = {"input": "\n".join(conversations)}

    return whisper_input


# Step 2: Use the Whisper API to generate a summary
def generate_summary_with_whisper(whisper_input):
    # API endpoint for Whisper summarization
    api_endpoint = "https://api.openai.com/v1/agents/whisper/summaries"

    # Set your OpenAI API key
    api_key = "your_openai_api_key"

    # Set request headers
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}

    # Send request to Whisper API
    response = requests.post(api_endpoint, json=whisper_input, headers=headers)

    # Check if request was successful
    if response.status_code == 200:
        summary = response.json()["summary"]
        return summary
    else:
        print("Error:", response.text)
        return None


# Step 3: Retrieve and process the summary generated by the Whisper API
def main(csv_file):
    # Step 1: Parse CSV file into Whisper API format
    whisper_input = parse_transcripts_to_whisper_format(csv_file)

    # Step 2: Use the Whisper API to generate a summary
    summary = generate_summary_with_whisper(whisper_input)

    # Step 3: Process and print the summary
    if summary:
        print("Summary:", summary)
    else:
        print("Failed to generate summary.")


# Replace 'your_openai_api_key' with your actual OpenAI API key
main("conversation_transcripts.csv")

# Start from here


In [1]:
import pandas as pd
import re
from transformers import pipeline

In [11]:
from datetime import datetime

current_timestamp = str.lower(datetime.now().strftime("%Y-%b-%d-%I-%M-%p"))
print(current_timestamp)

2024-feb-23-11-43-am


In [2]:
# Load your CSV file
data = pd.read_csv("data\Shark_Tank_US_Top_3_Products_For_Gamers_1.csv", index_col=0)
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
display(data.head())

# Extract the text from transcript
transcript_text = " ".join(
    data["text"].tolist()
)  # Replace "column_name" with your actual column name

newlines_reg = r"\n|\r|\r\n"  # Match `\n`, `\r`, or `\r\n`
transcript_text = re.sub(newlines_reg, " ", transcript_text)

print(transcript_text)

,speaker_label,start_time,end_time,text
0,spk_0,0.088483,0.136650,Into the tank is an accessory for the tech gen...
1,spk_1,0.318983,2.756150,Hi. My name is Joe Croft. Im from San Clemente...
2,spk_2,2.756333,2.864483,So yellow light is more calming and better for...
3,spk_1,2.864667,3.019983,much better and why blue light which is very c...
4,spk_0,3.024650,3.098317,Tell us about your sales. Theres gonna be a lo...


Into the tank is an accessory for the tech generation. Hi. My name is Joe Croft. Im from San Clemente California and Im here asking for $750,000 in exchange for 5% of my company, Gunner Optics Sharks. We live in a world dominated by screens, phones, computers, tablets, P CS. Youre probably staring at a screen 6 to 8 hours a day. At the end of a long day. Your eyes might feel a little bit like this, like theyre covered in sand or perhaps theyre dry and irritated because they take a pounding or maybe youve got a burning sensation from all that harsh blue light or maybe you just have a headache that wont go away. Computer eye strain has become the number one health related office complaint in the United States outstripping carpal tunnel by 40 to 1. Now, all those screens arent going away anytime soon. So, whats the solution? Gunner optics that um I work for a digital age. Our patented optical technology is the only clinically proven solution for relieving all four major symptoms of comput

In [3]:
import nltk  # For sentence segmentation

nltk.download("punkt")
print(nltk.data.path)  # Prints the NLTK data directory path

['C:\\Users\\41222/nltk_data', 'c:\\GitHub_Repos\\SartajBhuvaji\\Resonate\\Streamlit_App\\.venv\\nltk_data', 'c:\\GitHub_Repos\\SartajBhuvaji\\Resonate\\Streamlit_App\\.venv\\share\\nltk_data', 'c:\\GitHub_Repos\\SartajBhuvaji\\Resonate\\Streamlit_App\\.venv\\lib\\nltk_data', 'C:\\Users\\41222\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\41222\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
def chunk_text_preserving_sentences(text, max_chunk_size):
    """Chunks a given text while respecting sentence boundaries and combining
    sentences to meet the chunk size limit.

    Args:
        text (str): The input text to be chunked.
        max_chunk_size (int): The maximum allowed size for each chunk in characters.

    Returns:
        list: A list of text chunks, each respecting sentence boundaries
                and potentially consisting of combined sentences.
    """

    chunks = []
    current_chunk = ""
    sentence_tokenizer = nltk.sent_tokenize

    # Tokenize the text into sentences
    sentences = sentence_tokenizer(text)

    for sentence in sentences:
        # Calculate remaining space in the current chunk
        remaining_space = max_chunk_size - len(current_chunk)

        # If sentence fits within remaining space, add it directly
        if len(sentence) <= remaining_space:
            current_chunk += " " + sentence  # Add space between sentences
        else:
            # Sentence doesn't fit, check if combined with next sentence fits
            combined_sentence = current_chunk + " " + sentence
            if len(combined_sentence) <= max_chunk_size:
                # Combined sentence fits, add and clear current chunk
                chunks.append(combined_sentence)
                current_chunk = ""
            else:
                # Neither sentence nor combined version fit, add current chunk and
                # start a new one with the current sentence
                chunks.append(current_chunk)
                current_chunk = sentence

    # Add any remaining sentences or the final chunk if not empty
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

In [5]:
max_chunk_size = 1024

chunks = chunk_text_preserving_sentences(transcript_text, max_chunk_size)
display(chunks)

[' Into the tank is an accessory for the tech generation. Hi. My name is Joe Croft. Im from San Clemente California and Im here asking for $750,000 in exchange for 5% of my company, Gunner Optics Sharks. We live in a world dominated by screens, phones, computers, tablets, P CS. Youre probably staring at a screen 6 to 8 hours a day. At the end of a long day. Your eyes might feel a little bit like this, like theyre covered in sand or perhaps theyre dry and irritated because they take a pounding or maybe youve got a burning sensation from all that harsh blue light or maybe you just have a headache that wont go away. Computer eye strain has become the number one health related office complaint in the United States outstripping carpal tunnel by 40 to 1. Now, all those screens arent going away anytime soon. So, whats the solution? Gunner optics that um I work for a digital age. Our patented optical technology is the only clinically proven solution for relieving all four major symptoms of com

In [6]:
# Load an extractive summarization pipeline
extractive_summarizer_facebook_bart = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    max_length=150,
    min_length=30,
    do_sample=False,
)
extractive_summarizer_google_pegasus = pipeline(
    "summarization",
    model="google/pegasus-large",
    max_length=150,
    min_length=30,
    do_sample=False,
)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Process each chunk and combine the results
transcript_extractive_summarizer_facebook_bart = []
for chunk in chunks:
    summary = extractive_summarizer_facebook_bart(
        chunk,
    )[
        0
    ]["summary_text"]
    transcript_extractive_summarizer_facebook_bart.append(summary)

# Print the summary
print("summaries: ")
display(transcript_extractive_summarizer_facebook_bart)

Your max_length is set to 150, but your input_length is only 107. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=53)


summaries: 


['Computer eye strain has become the number one health related office complaint in the United States outstripping carpal tunnel by 40 to 1. Gunner optics is the only clinically proven solution for relieving all four major symptoms of computer eye strain.',
 'Gunner can help prevent dry eyes, reduce muscular, eye fatigue fight against glare. scientifically proven to reduce that harmful blue light emitted from those screens by 65%. Using your eye wear, youll find an increase in optical clarity, focus and visual performance.',
 'Last year, we sold $6.5 million worth of eyewear. We are approaching 60% gross margin. So we are going to build a brand. We have penetrated into the gaming category. I was thinking for gamers, this would be absolutely.',
 'E sports is the wild west right now. We have gone toe to toe with some of the largest people in the industry out there. We sponsor a number of E sports athletes.',
 'The company is profitable. We need to get the exposure out. We are evangelists 

In [8]:
# Process each chunk and combine the results
transcript_extractive_summarizer_google_pegasus = []
for chunk in chunks:
    summary = extractive_summarizer_google_pegasus(
        chunk,
    )[
        0
    ]["summary_text"]
    transcript_extractive_summarizer_google_pegasus.append(summary)

# Print the summary
print("summaries: ")
display(transcript_extractive_summarizer_google_pegasus)

Your max_length is set to 150, but your input_length is only 107. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=53)


summaries: 


['Computer eye strain has become the number one health related office complaint in the United States outstripping carpal tunnel by 40 to 1. Our patented optical technology is the only clinically proven solution for relieving all four major symptoms of computer eye strain.',
 'Gunner can help prevent dry eyes, reduce muscular, eye fatigue fight against glare and very importantly, scientifically proven to reduce that harmful blue light emitted from those screens by 65%. In fact, weve scientifically measured it, we block 65% of the blue light thats emitted from all those screens.',
 'So theres a lot of science that goes into why yellow light is much better and why blue light which is very close to the UV. Last year, we sold $6.5 million worth of eyewear. Youre doing a 60% margin, which is great, but you only made half a million E but wheres the cost going?',
 'E sports is the wild west right now. That is a great question because we not only have a patent, weve got battle tested. What happ

In [6]:
import dotenv
import os

dotenv.load_dotenv()

True

In [7]:
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
from docx import Document

In [8]:
def abstract_summary_extraction(transcription):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a highly skilled AI trained in language comprehension and summarization. I would like you to read the following text and summarize it into a concise abstract paragraph. Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text. Please avoid unnecessary details or tangential points.",
            },
            {"role": "user", "content": transcription},
        ],
    )
    return response.choices[0].message.content


def key_points_extraction(transcription):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a proficient AI with a specialty in distilling information into key points. Based on the following text, identify and list the main points that were discussed or brought up. These should be the most important ideas, findings, or topics that are crucial to the essence of the discussion. Your goal is to provide a list that someone could read to quickly understand what was talked about.",
            },
            {"role": "user", "content": transcription},
        ],
    )
    return response.choices[0].message.content


def action_item_extraction(transcription):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are an AI expert in analyzing conversations and extracting action items. Please review the text and identify any tasks, assignments, or actions that were agreed upon or mentioned as needing to be done. These could be tasks assigned to specific individuals, or general actions that the group has decided to take. Please list these action items clearly and concisely.",
            },
            {"role": "user", "content": transcription},
        ],
    )
    return response.choices[0].message.content


def meeting_minutes(transcription):
    abstract_summary = abstract_summary_extraction(transcription)
    key_points = key_points_extraction(transcription)
    action_items = action_item_extraction(transcription)
    return {
        "abstract_summary": abstract_summary,
        "key_points": key_points,
        "action_items": action_items,
    }

In [ ]:
# Process each chunk and combine the results
transcript_extractive_summarizer_openai_summary = []
transcript_extractive_summarizer_openai_key_points = []
transcript_extractive_summarizer_openai_action_items = []

for chunk in chunks[0]:
    summary, key_points, action_items = meeting_minutes(chunk)
    transcript_extractive_summarizer_openai_summary.append(summary)
    transcript_extractive_summarizer_openai_key_points.append(key_points)
    transcript_extractive_summarizer_openai_action_items.append(action_items)

# Print the summary
print("summaries: ")
display(transcript_extractive_summarizer_openai_summary)
print("key_points: ")
display(transcript_extractive_summarizer_openai_key_points)
print("action_items: ")
display(transcript_extractive_summarizer_openai_action_items)

In [20]:
print(transcript_extractive_summarizer_openai_summary)

['abstract_summary', 'abstract_summary', 'abstract_summary', 'abstract_summary', 'abstract_summary', 'abstract_summary', 'abstract_summary', 'abstract_summary', 'abstract_summary', 'abstract_summary']


In [ ]:
from transformers import BartTokenizer, BartModel

summarizer = pipeline(
    "summarization", model="t5-small", max_length=1024, min_length=30, do_sample=True
)